In [1]:
import sys
sys.path.append("./AIMA")
from search import *
from agents import*

In [2]:
class Tpiece(Thing): #Create a movable piece class
    pass

class Mobs(Thing):
    pass

class Oobs(Thing):
    pass

class Board(Environment): #Create an environment like last assignment
    pass

In [3]:
class PSA(SimpleProblemSolvingAgentProgram):
    def __call__(self, board): #AIMA overrided function because it had some things wrong
        self.percept = board.things #percept will just be the list of objects in the board environment
        self.state = self.update_state(self.percept)
        if not self.seq:
            goal = self.formulate_goal(self.state)
            problem = self.formulate_problem(self.state, goal)
            self.seq.append(self.search(problem))
            #This was originally self.seq = blabla in AIMA which obviously gives an error when trying to pop it later wtf AIMA
            if not self.seq:
                return None
        return self.seq.pop(0) #self.seq goes back empty for working the next goal
    
    def update_state(self, percept): #from board.things it builds a tuple of tuples to represent the initial state of the current step
        #Format: (('Identification char', (location)), next object of the percept in the same format)
        #Example: ( ('A', (1, 1)), ('T', (5, 2)) ) <- means a state with agent in 1, 1 and movable piece in 5, 2
        #Output needs to be tuple because the AIMA graph search function needs hashable data structures to add to the explored set
        
        state = {'M': [], 'O': []} #Easier to build the state as a dict
        for p in percept:
            if isinstance(p, Agent):
                state['A'] = p.location
            elif isinstance(p, Tpiece):
                state['T'] = p.location
            elif isinstance(p, Mobs):
                state['M'].append(p.location)
            elif isinstance(p, Oobs):
                state['O'].append(p.location)
            
        temp = {}
        temp['A'] = state['A']
        temp['T'] = state['T']
        temp['M'] = tuple(state['M'])
        temp['O'] = tuple(state['O'])
        
        state = []
        for k in temp.items():
            state.append(k)
                
        state = tuple(state) #convert the state to tuple
        return state

    def formulate_goal(self, state): #returns the desired state of the current step
        goal = [(('A', (4,5)), ('T', (5,5))), (('A', (5,4)), ('T', (5,5)))]
        #Ww gonna test doing everything in one step first
        return goal

    def formulate_problem(self, state, goal): #create a problem object with the initial state and goal state for this step
        problem = PushnPull(state, goal)
        return problem
    
    def search(self, problem): #Uses some search algorithm included in AIMA to solve the problem
        return graph_search(problem, [])
        

In [4]:
class PushnPull(Problem):
    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError
    
    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        temp = dict(state)
        #converting the state temporarily to a dictionary makes it easier to manip the data, example: {'A': (1,1), 'T': (5, 5)}
        
        if action == "pushRight":
            out = []
            for i in temp['M']:
                if temp['A'][0] == i[0] and temp['A'][1] + 1 == i[1]:
                    out.append((i[0], i[1] + 1))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] == temp['T'][0] and temp['A'][1] + 1 == temp['T'][1]:
                temp['T'] = (temp['T'][0], temp['T'][1] + 1)
            temp['A'] = (temp['A'][0], temp['A'][1] + 1) 
            
        elif action == "pushLeft":
            out = []
            for i in temp['M']:
                if temp['A'][0] == i[0] and temp['A'][1] - 1 == i[1]:
                    out.append((i[0], i[1] - 1))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] == temp['T'][0] and temp['A'][1] - 1 == temp['T'][1]:
                temp['T'] = (temp['T'][0], temp['T'][1] - 1)
            temp['A'] = (temp['A'][0], temp['A'][1] - 1)

        elif action == "pushUp":
            out = []
            for i in temp['M']:
                if temp['A'][0] - 1 == i[0] and temp['A'][1] == i[1]:
                    out.append((i[0] - 1, i[1]))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] - 1 == temp['T'][0] and temp['A'][1] == temp['T'][1]:
                temp['T'] = (temp['T'][0] - 1, temp['T'][1])
            temp['A'] = (temp['A'][0] - 1, temp['A'][1])

        elif action == "pushDown":
            out = []
            for i in temp['M']:
                if temp['A'][0] + 1 == i[0] and temp['A'][1] == i[1]:
                    out.append((i[0] + 1, i[1]))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] + 1 == temp['T'][0] and temp['A'][1] == temp['T'][1]:
                temp['T'] = (temp['T'][0] + 1, temp['T'][1])
            temp['A'] = (temp['A'][0] + 1, temp['A'][1])
            
        elif action == "pullRight":
            out = []
            for i in temp['M']:
                if temp['A'][0] == i[0] and temp['A'][1] - 1 == i[1]:
                    out.append((i[0], i[1] + 1))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] == temp['T'][0] and temp['A'][1] - 1 == temp['T'][1]:
                temp['T'] = (temp['T'][0], temp['T'][1] + 1)
            temp['A'] = (temp['A'][0], temp['A'][1] + 1)
            
        elif action == "pullLeft":
            out = []
            for i in temp['M']:
                if temp['A'][0] == i[0] and temp['A'][1] + 1 == i[1]:
                    out.append((i[0], i[1] - 1))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] == temp['T'][0] and temp['A'][1] + 1 == temp['T'][1]:
                temp['T'] = (temp['T'][0], temp['T'][1] - 1)
            temp['A'] = (temp['A'][0], temp['A'][1] - 1)
        
        elif action == "pullUp":
            out = []
            for i in temp['M']:
                if temp['A'][0] + 1 == i[0] and temp['A'][1] == i[1]:
                    out.append((i[0] - 1, i[1]))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] + 1 == temp['T'][0] and temp['A'][1] == temp['T'][1]:
                temp['T'] = (temp['T'][0] - 1, temp['T'][1])
            temp['A'] = (temp['A'][0] - 1, temp['A'][1])
            
        elif action == "pullDown":
            out = []
            for i in temp['M']:
                if temp['A'][0] - 1 == i[0] and temp['A'][1] == i[1]:
                    out.append((i[0] + 1, i[1]))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] - 1 == temp['T'][0] and temp['A'][1] == temp['T'][1]:
                temp['T'] = (temp['T'][0] + 1, temp['T'][1])
            temp['A'] = (temp['A'][0] + 1, temp['A'][1])
            
        else:
            if action == 'moveRight':
                temp['A'] = (temp['A'][0], temp['A'][1] + 1)
            elif action == 'moveLeft':
                temp['A'] = (temp['A'][0], temp['A'][1] - 1)
            elif action == 'moveUp':
                temp['A'] = (temp['A'][0] - 1, temp['A'][1])
            elif action == 'moveDown':
                temp['A'] = (temp['A'][0] + 1, temp['A'][1])
            #The new state has the agent move accordingly
        
        #This next block simply builds a proper tuple formatted state from the edited dictionary
        out = {}
        out['A'] = temp['A']
        out['T'] = temp['T']
        out['M'] = tuple(temp['M'])
        out['O'] = temp['O']
        
        result = []
        for k in out.items():
            result.append(k)
                
        result = tuple(result)
        return result
    
    
    def in_goal_state(state, goal):
        state = dict(state)
        agent = ('A', state['A'])
        treasure = ('T', state['T'])

        for g in goal:
            g_agent = g[0]
            g_treasure = g[1]
            if g_agent == agent and g_treasure == treasure:
                return True
        return False

In [5]:
def graph_search(problem, frontier): #This is just a default AIMA search algorithm which is ran from an agent's search method
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    If two paths reach a state, only use the first one. [Figure 3.7]"""
    frontier.append(Node(problem.initial))
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None